A script for making automated scanning jobs for the HP Envy 4500 printer.

Reverse-engineered the webscan service att http://192.168.1.17/#hId-pgWebScan

In [ ]:
import requests
import xml.etree.ElementTree as ET


In [ ]:
# printer ip address
PRINTER = "http://192.168.1.17"

cookies = {
    "sid": "s0538708d-811ba985e173db93169e975baf2840b7",
    "mobileView": "0",
}
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:101.0) Gecko/20100101 Firefox/101.0",
    "Accept": "application/xml, text/xml, */*",
    "Accept-Language": "en,sv-SE;q=0.8,sv;q=0.5,en-US;q=0.3",
    # 'Accept-Encoding': 'gzip, deflate',
    "DNT": "1",
    "Connection": "keep-alive",
    "Referer": PRINTER,
    # Requests sorts cookies= alphabetically
    # 'Cookie': 'sid=s0538708d-811ba985e173db93169e975baf2840b7; mobileView=0',
    "Pragma": "no-cache",
    "Cache-Control": "no-cache",
}



# Get scanner status

In [ ]:
response = requests.get(
    f"{PRINTER}/Scan/Status", headers=headers
)

root = ET.fromstring(response.content)
for child in root.iter():
    if child.tag.endswith("ScannerState"):
        break
else:
    print("Could not get ScannerState")

if child.text != "Idle":
    print(f"Scanner is not idle: {child.text}")


# Start scanning job

In [ ]:
dpi = 200
quality = 25

data = f"""
<scan:ScanJob xmlns:scan="http://www.hp.com/schemas/imaging/con/cnx/scan/2008/08/19" xmlns:dd="http://www.hp.com/schemas/imaging/con/dictionaries/1.0/" xmlns:fw="http://www.hp.com/schemas/imaging/con/firewall/2011/01/05">
    <scan:XResolution>{dpi}</scan:XResolution>
    <scan:YResolution>{dpi}</scan:YResolution>
    <scan:XStart>0</scan:XStart>
    <scan:YStart>0</scan:YStart>
    <scan:Width>2480</scan:Width>
    <scan:Height>3508</scan:Height>
    <scan:Format>Jpeg</scan:Format>
    <scan:CompressionQFactor>{quality}</scan:CompressionQFactor>
    <scan:ColorSpace>Color</scan:ColorSpace>
    <scan:BitDepth>8</scan:BitDepth>
    <scan:InputSource>Platen</scan:InputSource>
    <scan:GrayRendering>NTSC</scan:GrayRendering>
    <scan:ToneMap>
        <scan:Gamma>1000</scan:Gamma>
        <scan:Brightness>1000</scan:Brightness>
        <scan:Contrast>1000</scan:Contrast>
        <scan:Highlite>179</scan:Highlite>
        <scan:Shadow>25</scan:Shadow>
    </scan:ToneMap>
    <scan:ContentType>Photo</scan:ContentType>
</scan:ScanJob>
""".strip()

response = requests.post(
    f"{PRINTER}/Scan/Jobs",
    cookies=cookies,
    headers={"Content-Type": "text/xml", **headers},
    data=data,
)
response


# Get job list

In [ ]:
response = requests.get(f"{PRINTER}/Jobs/JobList", cookies=cookies, headers=headers)


In [ ]:
root = ET.fromstring(response.content)
print(root)


# Get job status

In [ ]:
response = requests.get(
    f"{PRINTER}/Jobs/JobList/1", cookies=cookies, headers=headers
)


# Get scanned page

In [ ]:
response = requests.get(
    f"{PRINTER}/Scan/Jobs/1/Pages/1", cookies=cookies, headers=headers
)